# Prepare NYC DNAInfo Data for Analysis

This notebook will document the procedure used to prepare the raw DNAInfo data for New York. Generally, the process employed aims to ensure that the resultant dataset contains valid data through the removal of spatially outlying and invalid polygons.

__Data Preparation__
1. Remove polygons outside of the New York metropolitan area.
2. Remove incomplete polygons (geometries that have no area - i.e. point or line geometries)
3. Fix 'sliver' polygons (these seem to derive from user issues with polygon closure).
4. Fix more complex geometry problems using convex hulls (these seem to stem from attempts to draw polygons with holes, or multiple polygons - neither is supported).
5. Remove remaining invalid polygons (i.e. invalid self-intersecting geometries that cannot be resolved).


In [2]:
import matplotlib.pyplot as plt
import geopandas as gpd
import pandas as pd
import shapely
from functools import partial
import numpy as np
from descartes import PolygonPatch
import Pycluster as pc
from itertools import combinations

%matplotlib inline

### Read in New York Data from combined geojson file
The neighbourhood drawings derive from the dataset compiled using the "ReadGeoms" Python notebook.

In [3]:
ny_data = r'C:\Users\djl543\OneDrive\Draw-Your-Neighborhood-master\NYC_raw_wgs84.geojson'
#ny_data = r'C:\Users\Dan\OneDrive\Draw-Your-Neighborhood-master\NYC_raw_wgs84.geojson'
ny_nhoods = gpd.read_file(ny_data)

# Set the crs of the input geojson to WGS84
ny_nhoods.crs = {'init': 'epsg:4326'}

# Tranform to projected coordinate system - EPSG:32118 - New York Long Island, NAD83-based projection in metres.
ny_nhoods = ny_nhoods.to_crs({'init':'epsg:32118'})

# Set Coordinate System
#ny_nhoods.crs = {'init': 'epsg:32118'}

print "Total records:" ,len(ny_nhoods)

42035


## Data Preparation

### Remove polygons outside of the New York metropolitan area
The 'New York area' is based on a shapefile that delineates the five boroughs of New York City: Brooklyn, The Bronx, Queens, Staten Island, and Manhattan. This shapefile is buffered by 10km, a size that aims to ensure that drawings of neighbourhoods within NYC that occur at the external boundary of the NYC area are definitely included. As such the buffer effectively mitigates against differences in precision between the authorititive and exact borough boundaries, and the contributed neighbourhood drawings.

This selection process also removes incomplete polygons (point or polygon geometries that intrinsically have no area), this is a technical property of difference in the spatial relationship between points, lines and polygons. 

In [4]:
# Read in New York City shapefile
nyc = gpd.read_file(r'C:\Users\djl543\OneDrive\Draw-Your-Neighborhood-master\NYC.shp')

# Get the envelope (bounding box) for the New York City area.
#ny_area = nyc['geometry'].unary_union.envelope

# Use a convex hull approach instead, 10km buffer adjusts gives leeway for drawings at borders.
ny_area = nyc['geometry'].unary_union.convex_hull.buffer(10000)

count_recs = len(ny_nhoods)

# Spatial Selection on the DNAInfo data, to be retained it must fall WITHIN the specified bounding box.
#out = ny_nhoods[~ny_nhoods.geometry.within(ny_area)]
ny_nhoods = ny_nhoods[ny_nhoods['geometry'].within(ny_area)]


print count_recs - len(ny_nhoods),"of", count_recs, "records removed"
print "Total records now:", len(ny_nhoods)

513 of 42035 records removed
Total records now: 41522


### Fix Invalid Polygons
The dataset of neighbourhood drawings contains around 2,000 polygon geometries that are invalid owing to self-intersecting edges. Invalid polygon geometries give rise to incorrect estimates of polygon area and centroid location, and can potentially lead to incorrect conclusions being drawn regarding their spatial relationship with other features. As such their inclusion has the potential to bias any analysis undertaken. Rather than immediately remove these invalid drawings, we first try and fix the invalid polygons or try to derive a valid 'representative' polygon based on the invalid polygon. Generally, invalid neighborhood polygon geometries can be grouped into two ad hoc categories: polygon closure issues; and, more complex geometrical issues.

Polygon closure issues arise when a contributor attempt to close a polygon having drawn a neighbourhood shape, but in the process accidentally and often unknowingly creates a tiny spur or 'sliver' polygon in addition to the main neighbourhood drawing by crossing over the initial edge of the neighbourhood drawing. We fix these by algorithmically spliting the invalid geometry into two polygons, and then removing the small sliver geometry. These are generally tiny polygons, and we set an arbitrary threshold for removal such that to be considered a sliver the candidate polygon can only be 2% or less of the area of the main polygon geometry. Invalid polygon geometries that can be fixed using this procedure account for c.55% of all invalid polygons.

More complex geometrical issues tend to arise when contributors attempt to draw multiple regions, or to draw polygons with holes in them - perhaps to exclude a major road or to account for a natural break, body of water, industrial site etc. These polygons cannot be fixed by simply removing parts of them, as this would result in a substantial amount of the area covered by the polygon being lost. Instead the polygon convex hull is used as a representative polygon, based on the ratio of the area of the convex hull to the set of poylgons that characterise the invalid polygon geometry. In a few cases, the convex hull is itself a good representative polygon, however in most cases we create a sub-polygon of the convex hull based on the polygon difference with the original invalid polygon. We iteratively test the representativeness of the convex hull and convex hull sub-polygons down to an area ratio of 95%. This second process allows for the inclusion of representative polygons for another c.22% of the invalid polygons.

The remaining c.23% of poylgons with invalid geometries are deemed ot be unfixable and removed from the dataset. They tend toward two cases - the classic 'bow tie' shaped polygon, and the extremely complex overlapping polygon reminiscent of a scribble. 

In [5]:
def makeMultipoly(poly):
    # Convert polygon boundaries to linear rings which account for intersecting loops
    ring = shapely.ops.unary_union(poly.boundary)
    return shapely.geometry.MultiPolygon(list(shapely.ops.polygonize(ring)))

def getBigPoly(poly):
    if len(poly.geoms) > 1:
        big_poly = g.geoms[0]
        for i in range(1,len(poly.geoms)):
            if poly.geoms[i].area > big_poly.area:
                big_poly = poly.geoms[i]
    else:
        big_poly = poly.geoms[0]
    return big_poly

def subPolyOne(poly,convex):
    diff = convex.difference(poly)
    # Make each difference a 'solid' polygon without holes
    if type(diff) == shapely.geometry.polygon.Polygon:
        x,y = diff.exterior.xy
        points = zip(x,y)
        diff = shapely.geometry.Polygon((points))
    else:
        temp = []
        for geo in diff:
            x,y = geo.exterior.xy
            points = zip(x,y)
            temp.append(shapely.geometry.Polygon((points)))
        diff = shapely.geometry.MultiPolygon(temp)
    
    # Now find the largest member of diff and produce a subpolygon of the convex hull
    if type(diff) == shapely.geometry.polygon.Polygon:
        return convex.difference(diff)     
    else:
        best = [convex.difference(diff.geoms[0]),(poly.area/convex.difference(diff.geoms[0]).area)*100]
        for i in range(1,len(diff.geoms)):
            if abs(((poly.area/convex.difference(diff.geoms[i]).area)*100)-100) < abs(best[1] -100):
                best = [convex.difference(diff.geoms[i]),(poly.area/convex.difference(diff.geoms[i]).area)*100]
        return best[0]
    
def subPolyTwo(poly,convex):
    diff = convex.difference(poly)
    # Make each difference a 'solid' polygon without holes
    if type(diff) == shapely.geometry.polygon.Polygon:
        x,y = diff.exterior.xy
        points = zip(x,y)
        diff = shapely.geometry.Polygon((points))
    else:
        temp = []
        for geo in diff:
            x,y = geo.exterior.xy
            points = zip(x,y)
            temp.append(shapely.geometry.Polygon((points)))
        diff = shapely.geometry.MultiPolygon(temp)
    
    # If the difference only produces a single polygon, return that as in subPolyOne.
    if type(diff) == shapely.geometry.polygon.Polygon:
        return convex.difference(diff) 
    
    # Otherwise find the two members of diff that produces the best subpolygon of the convex hull
    valid_polygons = []
    for i,j in combinations(range(0,len(diff.geoms)),2):
        # This first bit creates a candidate dataset to evaluate based on valid combinations of polygons.
        testpoly = shapely.geometry.MultiPolygon([diff.geoms[i],diff.geoms[j]])
        testpoly = convex.difference(testpoly)
        if testpoly.is_valid and type(testpoly) == shapely.geometry.polygon.Polygon:
            valid_polygons.append(testpoly)
    # Now inspect the candidates and return the best, if available
    if len(valid_polygons) == 0:
        # If there are no valid options, just return the convex hull
        return convex
    elif len(valid_polygons) == 1:
        # If there is 1 valid option, return that.
        return valid_polygons[0]
    else:
        # Otherwise, work out which combo is closest in area to the actual shape and return that.
        best2 = [valid_polygons[0],(poly.area/valid_polygons[0].area)*100]
        for i in range(1,len(valid_polygons)):
            if abs(((poly.area/valid_polygons[i].area)*100)-100) < abs(best2[1] -100):
                best2 = [valid_polygons[i],(poly.area/valid_polygons[i].area)*100]
        return best2[0]

def compareAreas(poly1,poly2,pc):
    if (poly1.area/poly2.area)*100 >= float(pc):
        return True
    else:
        return False

update_polys = []
check_polys = []
remove_polys = []
for row in ny_nhoods.iterrows():
    if not row[1]['geometry'].is_valid:
        # First create a new valid multipolygon geometry for these invalid polygons.
        g = makeMultipoly(row[1]['geometry'])
        
        # Get the biggest polygon from the new MultiPolygon - effectively disregard sliver polygons.
        bigPoly = getBigPoly(g)
        
        # Check if bigPoly makes up 98%+ of the total area of the original and pass to update list if so.
        if compareAreas(bigPoly,g,98):
            update_polys.append([row[0],"geometry",bigPoly])
            check_polys.append(row[1]['shapeID'])
            continue
       
        # Now consider buffering-based solutions to remove internal loops
        # NB we buffer both the Polygon and MultiPolygon geometries due to different behaviour
        buff = row[1]['geometry'].buffer(0)
        buffg = g.buffer(0)
        
        # Check if the buffer outputs make up 98%+ of the total area of the original and pass to update list if so.
        if compareAreas(buff,g,98) and type(buff) == shapely.geometry.polygon.Polygon:
            update_polys.append([row[0],"geometry",buff])
            check_polys.append(row[1]['shapeID'])
            continue
        if compareAreas(buffg,g,98) and type(buffg) == shapely.geometry.polygon.Polygon:
            update_polys.append([row[0],"geometry",buffg])
            check_polys.append(row[1]['shapeID'])
            continue
        
        # Now try a basic convex hull solution
        convexh = g.convex_hull
        
        # Check if the convex hull makes up 98% of the total area of the original and pass to update list if so.
        # NB convex hull goes second for comparison as our expectation is that it is bigger.
        if compareAreas(g,convexh,98):
            update_polys.append([row[0],"geometry",convexh])
            check_polys.append(row[1]['shapeID'])
            continue
        
        # Now try the 1st sub-polygon of the convex hull.
        sub1 = subPolyOne(g,convexh)
        
        # Check if the first subpolygon makes up 98% of the total area of the original and pass to update list if so.
        # NB first sub polygon goes second for comparison as our expectation is that it is bigger (or exactly the same size).
        if compareAreas(g,sub1,98) and type(sub1) == shapely.geometry.polygon.Polygon:
            update_polys.append([row[0],"geometry",sub1])
            check_polys.append(row[1]['shapeID'])
            continue
        
        # Finally try the 2nd sub-polygon of the convex hull.
        sub2 = subPolyTwo(g,convexh)
        
        # Check if the second subpolygon makes up 98% of the total area of the original and pass to update list if so.
        if compareAreas(g,sub2,98) and type(sub2) == shapely.geometry.polygon.Polygon:
            update_polys.append([row[0],"geometry",sub2])
            check_polys.append(row[1]['shapeID'])
            continue
        
        # Finally, look at incrementally relaxing the convex hull approach down to 95%
        flag = 0
        for thresh in [97,96,95]:
            if compareAreas(g,sub1,thresh) and flag == 0:
                update_polys.append([row[0],"geometry",sub1])
                check_polys.append(row[1]['shapeID'])
                flag = 1
            elif compareAreas(g,sub2,thresh) and flag == 0:
                update_polys.append([row[0],"geometry",sub2])
                check_polys.append(row[1]['shapeID'])
                flag = 1  
        if flag == 1:
            continue
        
        remove_polys.append(row[1]['shapeID'])

# Get original polygons that were fixed
orig_fix = ny_nhoods[ny_nhoods['shapeID'].isin(check_polys)]

# Update the rows with their fixed polygon geometries
for index, col, value in update_polys:
    ny_nhoods = ny_nhoods.set_value(index,col,value)

# Get removed polygons for additional consideration.
bad_geoms = ny_nhoods[ny_nhoods['shapeID'].isin(remove_polys)]

# Get fixed polygons for additional consideration
fix = ny_nhoods[ny_nhoods['shapeID'].isin(check_polys)]
    
# Delete the invalid geometry polygons that we don't think are slivers.
ny_nhoods = ny_nhoods[~ny_nhoods['shapeID'].isin(remove_polys)]

print len(update_polys), "polygons were fixed"
print ""
print len(remove_polys), "polygons couldn't be adequately fixed and were removed"
print ""
print "Total records now:", len(ny_nhoods)


Self-intersection at or near point 305876.38417153212 76666.669419157522
Self-intersection at or near point 312237.67131414998 77967.317314318469
Self-intersection at or near point 311783.94769788359 77231.892729715197
Self-intersection at or near point 300396.82174327294 60778.415920446787
Self-intersection at or near point 301382.15427476732 63454.861298941621
Self-intersection at or near point 301706.37981041934 61295.218007282558
Self-intersection at or near point 284917.81924101856 43196.682898855302
Self-intersection at or near point 286583.24444313813 40390.588387449083
Self-intersection at or near point 284511.8809964516 42059.972700751023
Self-intersection at or near point 284609.67909865238 40399.017048533766
Self-intersection at or near point 282940.97960421618 41411.070661137812
Self-intersection at or near point 285767.52344170713 41431.07011326107
Self-intersection at or near point 284569.68875776022 42218.950311040142
Self-intersection at or near point 282848.29333444987

1504 polygons were fixed

425 polygons couldn't be adequately fixed and were removed

Total records now: 41097


In [6]:
# As this is all the data preparation we will do, we can export a copy of the prepared data.
# This data is not subject to any rules regarding removal of extreme values.
# However it is subject to the threshold value for invalid polygons (95%+) and the 'New York' buffer size.
# NB we convert the geojson to WGS84 in light of the geojson standard.
ny_nhoods = ny_nhoods.to_crs({'init':'epsg:4326'})
ny_nhoods.to_file('NYC_prep_wgs84.geojson',driver='GeoJSON')
#ny_nhoods.to_file('NYC_prep_wgs84.shp',driver='ESRI Shapefile')

NB: The dataset that this script produces is effectively analysis ready, however, it makes no attempt to remove disjoint neighbourhood boundaries, or boundaries that deviate from their group singificantly in terms of their size or shape.